In [1]:
import pandas as pd
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
# Load the preprocessed dataset
df = pd.read_csv('preprocessed_train.csv')

# Drop rows with NaN values in the 'preprocessed_review' column
df = df.dropna(subset=['preprocessed_review'])

# Convert ratings to one-hot encoded labels
labels = to_categorical(df['rating'] + 1)  # Adding 1 to convert -1, 0, 1 to 0, 1, 2
print(labels, labels[0])
# Tokenize the Arabic text
tokenizer_arabic = Tokenizer()
tokenizer_arabic.fit_on_texts(df['preprocessed_review'])
sequences_arabic = tokenizer_arabic.texts_to_sequences(df['preprocessed_review'])
max_sequence_length = 100  # Set your desired sequence length
padded_sequences_arabic = pad_sequences(sequences_arabic, maxlen=max_sequence_length)



[[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]] [1. 0. 0.]


In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import pandas as pd

# Load and preprocess your data
# Assuming df is your preprocessed dataset with columns: 'preprocessed_review' and 'rating'

# Load the preprocessed dataset
df = pd.read_csv('preprocessed_train.csv')

# Drop rows with NaN values in the 'preprocessed_review' column
df = df.dropna(subset=['preprocessed_review'])

# Convert ratings to categorical labels
labels = to_categorical(df['rating'] + 1)  # Adding 1 to convert -1, 0, 1 to 0, 1, 2

# Tokenize the preprocessed reviews
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['preprocessed_review'])
sequences = tokenizer.texts_to_sequences(df['preprocessed_review'])
max_sequence_length = 100  # Set your desired sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Define the BiLSTM model
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))  # BiLSTM layer
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=32)))  # BiLSTM layer
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Output layer with 3 classes

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Display the model summary
model.summary()

# Train the model with your padded_sequences and labels
model.fit(padded_sequences, labels, epochs=10, batch_size=32, validation_split=0.2)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          3003520   
                                                                 
 bidirectional (Bidirection  (None, 100, 128)          98816     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 100, 128)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2

773/773 [==============================] - 114s 134ms/step - loss: 0.5329 - accuracy: 0.8020 - val_loss: 0.4726 - val_accuracy: 0.8390
Epoch 2/10
773/773 [==============================] - 159s 206ms/step - loss: 0.3599 - accuracy: 0.8768 - val_loss: 0.4793 - val_accuracy: 0.8275
Epoch 3/10
773/773 [==============================] - 162s 209ms/step - loss: 0.2740 - accuracy: 0.9054 - val_loss: 0.5271 - val_accuracy: 0.8175
Epoch 4/10
773/773 [==============================] - 162s 209ms/step - loss: 0.2172 - accuracy: 0.9269 - val_loss: 0.5887 - val_accuracy: 0.8108
Epoch 5/10
773/773 [==============================] - 164s 213ms/step - loss: 0.1770 - accuracy: 0.9423 - val_loss: 0.6449 - val_accuracy: 0.8081
Epoch 6/10
773/773 [==============================] - 163s 211ms/step - loss: 0.1480 - accuracy: 0.9517 - val_loss: 0.7066 - val_accuracy: 0.7979
Epoch 7/10
773/773 [==============================] - 164s 213ms/step - loss: 0.1262 - accuracy: 0.9581 - val_loss: 0.7592 - val_accura

In [3]:
# show the accuracy of the model
loss, accuracy = model.evaluate(padded_sequences_arabic, labels, verbose=False)

print("Training Accuracy: {:.4f}".format(accuracy))

Training Accuracy: 0.9388
